In [42]:
from sympy import symbols, init_printing
import sympy
import sympy.physics.mechanics as me
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
# import the ode ODE solver
from scipy.integrate import ode
%matplotlib inline
# init_printing(False)
init_printing(True)
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

In [14]:
# Define the state variables and their derivatives
x1, x2, theta1, theta2 = me.dynamicsymbols('x_1 x_2 theta_1 theta2')
x1_dot, x2_dot = me.dynamicsymbols('x_1 x_2', 1)
theta1_dot, theta2_dot = me.dynamicsymbols('theta_1 theta2', 1)
f1, f2 = me.dynamicsymbols('f1 f2')

# Define the constants
cart1_mass, cart2_mass, m1, m2 = sympy.symbols('cart1_mass cart2_mass m1 m2')
M = sympy.symbols('M')
g, L1, L2, L3, t, I = sympy.symbols('g L1 L2 L3 t I')

In [35]:
# Define the Outer fixed Frame
A = me.ReferenceFrame('A')

# Define the first pendulum frame
B = A.orientnew('B', 'Axis', [theta1, A.z])

# Define the second pendulum frame
C = A.orientnew('C', 'Axis', [theta2, A.z])

# The cart points and velocities
C1 = me.Point('C1')
C2 = me.Point('C2')
C1.set_pos(C2, (x2 - x1) * A.x )
C1.set_vel(A, x1_dot * A.x)
C2.set_vel(A, x2_dot * A.x)

# Create Particles of the carts
Cart1 = me.Particle('Cart1', C1, cart1_mass)
Cart2 = me.Particle('Cart2', C2, cart2_mass)

# The hook point locations
P1 = C1.locatenew('P1', -L1 * B.y)
P2 = C2.locatenew('P2', -L2 * C.y)

# Create the L3 relation
P2_vector_P1 = P2.pos_from(P1).normalize()
P1.set_pos(P2, P2_vector_P1 * L3)

# Set the hook velocities
P1.v2pt_theory(C1, A, B)
P2.v2pt_theory(C2, A, C)

# Create the hook particles
hook1 = me.Particle('hook1', P1, m1)
hook2 = me.Particle('hook2', P2, m2)

# Create the Payload center of mass
P3 = P1.locatenew('P3', P2_vector_P1 * (L3/2))
P3.v2pt_theory(P2, A, A)

# Create the plate inertial tensor
I_Payload = me.inertia(A, 0, 0, I)
inertia_Payload = (I_Payload, P3)

# Create the Payload Rigid Body
Payload = me.RigidBody('Payload', P3, A, M, inertia_Payload)

In [36]:
hook1.potential_energy = (-m1 * g * L1 * sympy.cos(theta1))
hook2.potential_energy = (-m2 * g * L2 * sympy.cos(theta2))
Payload.potential_energy = (-M * g * L3 * (C1.pos_from(P3).express(A) & A.y))

In [41]:
forces = [(C1, f1 * A.x),
          (C2, f2 * A.x)]

# Form the Lagrangian, then simplify and print
L = me.Lagrangian(A, hook1, hook2, Payload)
LM = me.LagrangesMethod(L, [x1, x2, theta1, theta2],
                        forcelist = forces, frame = A)
LM.form_lagranges_equations()
lrhs = LM.rhs()

In [45]:
# define the forcing function
def force(t):
    # set the maximum force output
    fmax = 10.0
    
    # return a bang-bang command
    return fmax * ((t > 0.25) - 2 * (t >= 1.25) + (t >= 2.25))

In [49]:
# Define the states and state vector
w1, w2, w3, w4, w5, w6, w7, w8 = sympy.symbols('w1 w2 w3 w4 w5 w6 w7 w8',
                                               cls=sympy.Function)
w = [w1(t), w2(t), w3(t), w4(t), w5(t), w6(t), w7(t), w8(t)]

# Set up the state definitions and parameter substitution
sub_params = {x1 : w1(t),
              x2 : w2(t), 
              theta1 : w3(t),
              theta2 : w4(t),
              x1_dot : w5(t),
              x2_dot : w6(t), 
              theta1_dot : w7(t),
              theta2_dot : w8(t),
              cart1_mass: 100,
              cart2_mass: 100,
              m1: 2,
              m2: 2,
              M: 1,
              g: 9.81, 
              L1: 2, 
              L2: 1, 
              L3: 3,
              I: (1 * 3**2) / 12,
              f1: force(t),
              f2: force(t)}

# set this parameter to enable array output from sympy.lambdify
mat2array = [{'ImmutableMatrix': np.array}, 'numpy']

# Create a function from the equations of motion
# Here, we substitude the states and parameters as appropriate prior to the lamdification
eq_of_motion = sympy.lambdify((t, w), 
                              lrhs.subs(sub_params), 
                              modules = mat2array)

# Set up the initial conditions for the solver
x1_init = 0.0                   # Initial trolley position (m)
x2_init = 2.8284
x1_dot_init = 0.0               # Initial trolley velocity (m/s)
x2_dot_init = 0.0
theta1_init = 0.0 * np.pi/180   # Initial angle (rad)
theta2_init = 0.0 * np.pi/180
theta1_dot_init = 0.0           # Initial angular velocity (rad/s)
theta2_dot_init = 0.0  

# Pack the initial conditions into an array
x0 = [x1_init,
      x2_init,
      theta1_init,
      theta2_init,
      x1_dot_init,
      x2_dot_init,
      theta1_dot_init,
      theta2_dot_init]

# Create the time samples for the output of the ODE solver
sim_time = np.linspace(0.0, 10.0, 1001) # 0-10s with 1001 points in between

In [50]:
# Set up the initial point for the ode solver
r = ode(eq_of_motion).set_initial_value(x0, sim_time[0])
 
# define the sample time
dt = sim_time[1] - sim_time[0]   

# pre-populate the response array with zeros
response = np.zeros((len(sim_time), len(x0)))

# Set the initial index to 0
index = 0

# Now, numerically integrate the ODE while:
#   1. the last step was successful
#   2. the current time is less than the desired simluation end time
while r.successful() and r.t < sim_time[-1]:
    response[index, :] = r.y
    r.integrate(r.t + dt)
    index += 1

//anaconda/lib/python3.5/site-packages/numpy/__init__.py:1: RuntimeWarning: divide by zero encountered in double_scalars
  """
//anaconda/lib/python3.5/site-packages/scipy/integrate/_ode.py:869: UserWarning: vode: Repeated convergence failures. (Perhaps bad Jacobian supplied or wrong choice of MF or tolerances.)
  'Unexpected istate=%s' % istate))
